# Figure S1

Supplementary figure 1: Response to elevated glucose concentrations in steps.

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: t, Glc, G3P, Pyr, NADH_c, NADH_m, Ca_c, Ca_m, ΔΨm, ATP_c, ADP_c, AMP_c, degavg, x
using MitochondrialDynamics.Utils: second, μM, mV, mM, Hz

import PyPlot as plt
rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["font.size"] = 14

In [ ]:
function glc_step(t)
    return 5.0mM * (1 + (t >= 1000.0second) + (t >= 2000.0second))
end

tend = 3000.0second
@named sys = make_model(; glcrhs=glc_step(t))
prob = ODEProblem(sys, [], tend)
sol = solve(prob, tstops=[1000.0second, 2000.0second]);

In [ ]:
function plot_figs1(
    sol;
    figsize=(10, 10),
    tspan=(0.0, 3000.0second),
    density::Int=101,
    tight=true
)
    ts = LinRange(tspan[1], tspan[2], density)
    g3p = sol.(ts, idxs=G3P) .* 1000
    pyr = sol.(ts, idxs=Pyr) .* 1000
    nadh_c = sol.(ts, idxs=NADH_c) .* 1000
    nadh_m = sol.(ts, idxs=NADH_m) .* 1000
    ca_c = sol.(ts, idxs=Ca_c) .* 1000
    ca_m = sol.(ts, idxs=Ca_m) .* 1000
    atp_c = sol.(ts, idxs=ATP_c) .* 1000
    adp_c = sol.(ts, idxs=ADP_c) .* 1000
    amp_c = sol.(ts, idxs=AMP_c) .* 1000
    dpsi = sol.(ts, idxs=ΔΨm) .* 1000
    k = sol.(ts, idxs=degavg)
    x1 = sol.(ts, idxs=x[1])
    x2 = sol.(ts, idxs=x[2])
    x3 = sol.(ts, idxs=x[3])

    fig, ax = plt.subplots(3, 3; figsize)

    ax[1, 1].plot(ts, g3p)
    ax[1, 1].set(title="(A) G3P (μM)", ylim=(0.0, 8.0))

    ax[1, 2].plot(ts, pyr)
    ax[1, 2].set(title="(B) Pyruvate (μM)", ylim=(0.0, 80.0))

    ax[1, 3].plot(ts, ca_c, label="cyto")
    ax[1, 3].plot(ts, ca_m, label="mito")
    ax[1, 3].set(title="(C) Calcium (μM)")
    ax[1, 3].legend()

    ax[2, 1].plot(ts, nadh_c, label="cyto")
    ax[2, 1].plot(ts, nadh_m, label="mito")
    ax[2, 1].set(title="(D) NADH (μM)")
    ax[2, 1].legend()

    ax[2, 2].plot(ts, atp_c, label="ATP")
    ax[2, 2].plot(ts, adp_c, label="ADP")
    ax[2, 2].plot(ts, amp_c, label="AMP")
    ax[2, 2].set(title="(E) Adenylates (μM)")
    ax[2, 2].legend()

    ax[2, 3].plot(ts, atp_c ./ adp_c)
    ax[2, 3].set(title="(F) ATP:ADP", ylim=(0, 40))

    ax[3, 1].plot(ts, dpsi)
    ax[3, 1].set(title="(G) ΔΨ (mV)", ylim=(80, 150), xlabel="Time (seconds)")

    ax[3, 2].plot(ts, x1, label="X1")
    ax[3, 2].plot(ts, x2, label="X2")
    ax[3, 2].plot(ts, x3, label="X3")
    ax[3, 2].set(title="(H) Mitochondrial nodes", xlabel="Time (seconds)")
    ax[3, 2].legend()

    ax[3, 3].plot(ts, k)
    ax[3, 3].set(title="(I) Average Node Degree", xlabel="Time (seconds)")

    for a in ax
        a.grid()
    end

    fig.set_tight_layout(tight)
    return fig
end

In [ ]:
figs1 = plot_figs1(sol);

In [ ]:
# Uncomment if pdf file is required
# figs1.savefig("FigS1.tif", dpi=300, format="tiff", pil_kwargs=Dict("compression" => "tiff_lzw"))